In [1]:
import numpy as np

In [2]:
def calculate_profit(x1,x2,y,z,n0,p1,p2,p_product):
    #参数说明
    #n:零件1的数量/零件2的数量
    #p1:零件1的次品率
    #p2:零件2的次品率
    #p_product:成品的次品率
    #一些内置的参数
    C1=2#检测零件1的成本
    C2=3#检测零件2的成本
    C_market=56#单件成品的市场售价
    C_assemble=6#装配单件成品的费用
    C_test_product=3#检测单件成品所需的费用
    C_lose=30#调换一件产品引起的损失
    C_disassemble=5#拆解一件产品所需的费用
    #更新次品率
    p1_update=(1-x1)*p1
    p2_update=(1-x2)*p2
    #成品的总次品率
    P=(1-p1_update)*(1-p2_update)*p_product+p1_update+p2_update-p1_update*p2_update
    n=np.floor(min((1-x1*p1)*n0,(1-x2*p2)*n0))
    profit=C_market*(1-P)*n-x1*C1*n0-x2*C2*n0-C_assemble*n-y*C_test_product*n-(1-y)*C_lose*n*P-z*C_disassemble*n*P
    #更新零件数
    if z==0:
        n_update=0
    elif z==1:
        n_update=np.floor(n*P)
    return profit,n_update,p1_update,p2_update

In [3]:
n0=10000
p1=0.1
p2=0.1
p_product=0.1
S=0
same=0
different=0
for p1 in np.linspace(0.95*p1,1.05*p1,5):
    for p2 in np.linspace(0.95*p2,1.05*p2,5):
        for p_product in np.linspace(0.95*p_product,1.05*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,1,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,1,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[1,1,0,0]):
                same+=1
            else:
                different+=1

1
最终的最大利润为43128827.23311927
第一阶段的生产决策为:
[0, 0, 0, 1, 264508.03875, 2587.0, 0.095, 0.095]
第二阶段的生产决策为:
[0, 0, 0, 1, 68428.22962462499, 669.0, 0.095, 0.095]
第三阶段的生产决策为:
[1, 1, 0, 0, 21962.15, 0, 0.0, 0.0]
2
最终的最大利润为43568204.670770094
第一阶段的生产决策为:
[0, 0, 0, 1, 262644.75687499996, 2608.0, 0.095, 0.095]
第二阶段的生产决策为:
[0, 0, 0, 1, 68497.75259300001, 680.0, 0.095, 0.095]
第三阶段的生产决策为:
[1, 1, 0, 0, 22193.225, 0, 0.0, 0.0]
3
最终的最大利润为43929540.049450114
第一阶段的生产决策为:
[0, 0, 0, 1, 260781.47499999992, 2628.0, 0.095, 0.095]
第二阶段的生产决策为:
[0, 0, 0, 1, 68533.37162999998, 690.0, 0.095, 0.095]
第三阶段的生产决策为:
[1, 1, 0, 0, 22383.6, 0, 0.0, 0.0]
4
最终的最大利润为44443294.83350715
第一阶段的生产决策为:
[0, 0, 0, 1, 258918.19312500005, 2649.0, 0.095, 0.095]
第二阶段的生产决策为:
[0, 0, 0, 1, 68587.42935881251, 701.0, 0.095, 0.095]
第三阶段的生产决策为:
[1, 1, 1, 0, 22653.84, 0, 0.0, 0.0]
5
最终的最大利润为44951212.648439296
第一阶段的生产决策为:
[0, 0, 0, 1, 257054.91124999998, 2669.0, 0.095, 0.095]
第二阶段的生产决策为:
[0, 0, 0, 1, 68607.955812625, 712.0, 0.095, 0.095]
第三阶段的生产决策为:
[

In [4]:
print(f"在0.95-1.05波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.95-1.05波动时,决策差异率为:{different/125}")

在0.95-1.05波动时,生产决策方案和原方案一致的次数为:0
在0.95-1.05波动时,生产决策方案和原方案不一致的次数为:125
在0.95-1.05波动时,决策差异率为:1.0


In [5]:
n0=10000
p1=0.1
p2=0.1
p_product=0.1
S=0
same=0
different=0
for p1 in np.linspace(0.9*p1,1.1*p1,5):
    for p2 in np.linspace(0.9*p2,1.1*p2,5):
        for p_product in np.linspace(0.9*p_product,1.1*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,1,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,1,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[1,1,0,0]):
                same+=1
            else:
                different+=1

1
最终的最大利润为39952170.76856003
第一阶段的生产决策为:
[0, 0, 0, 1, 275749.61, 2464.0, 0.09000000000000001, 0.09000000000000001]
第二阶段的生产决策为:
[0, 0, 0, 1, 67944.703904, 607.0, 0.09000000000000001, 0.09000000000000001]
第三阶段的生产决策为:
[1, 1, 0, 0, 20292.52, 0, 0.0, 0.0]
2
最终的最大利润为40742883.69941259
第一阶段的生产决策为:
[0, 0, 0, 1, 271981.75499999995, 2505.0, 0.09000000000000001, 0.09000000000000001]
第二阶段的生产决策为:
[0, 0, 0, 1, 68131.42962749998, 627.0, 0.09000000000000001, 0.09000000000000001]
第三阶段的生产决策为:
[1, 1, 0, 0, 20708.1, 0, 0.0, 0.0]
3
最终的最大利润为41703849.210550025
第一阶段的生产决策为:
[0, 0, 0, 1, 268213.9, 2547.0, 0.09000000000000001, 0.09000000000000001]
第二阶段的生产决策为:
[0, 0, 0, 1, 68314.08033, 648.0, 0.09000000000000001, 0.09000000000000001]
第三阶段的生产决策为:
[0, 0, 1, 0, 21213.08352, 0, 0.09000000000000001, 0.09000000000000001]
4
最终的最大利润为42716059.96523008
第一阶段的生产决策为:
[0, 0, 0, 1, 264446.0449999999, 2588.0, 0.09000000000000001, 0.09000000000000001]
第二阶段的生产决策为:
[0, 0, 0, 1, 68438.63644599999, 669.0, 0.09000000000000001, 0.0900000

In [6]:
print(f"在0.9-1.1波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.9-1.1波动时,决策差异率为:{different/125}")

在0.9-1.1波动时,生产决策方案和原方案一致的次数为:0
在0.9-1.1波动时,生产决策方案和原方案不一致的次数为:125
在0.9-1.1波动时,决策差异率为:1.0


In [7]:
n0=10000
p1=0.1
p2=0.1
p_product=0.1
S=0
same=0
different=0
for p1 in np.linspace(0.8*p1,1.2*p1,5):
    for p2 in np.linspace(0.8*p2,1.2*p2,5):
        for p_product in np.linspace(0.8*p_product,1.2*p_product,5):
            S+=1
            #第一阶段的生产决策
            result=[]
            for x1 in [0,1]:
                for x2 in [0,1]:
                    for y in [0,1]:
                        for z in [0,1]:
                            profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,n0,p1,p2,p_product)
                            result.append([x1,x2,y,z,profit,n_update,p1_update,p2_update])
            #第二阶段的决策
            result1=np.empty((8, 16), dtype=object)
            for i in range(1,16,2):
                count=0
                for x1 in [0,1]:
                    for x2 in [0,1]:
                        for y in [0,1]:
                            for z in [0,1]:
                                profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result[i][5],result[i][6],result[i][7],p_product)
                                result1[int((i-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                count+=1
            #第三阶段决策
            result2=np.empty((8,8,16), dtype=object)
            for i in range(8):
                for j in range(1,16,2):
                    count=0
                    for x1 in [0,1]:
                        for x2 in [0,1]:
                            for y in [0,1]:
                                for z in [0,1]:
                                    profit,n_update,p1_update,p2_update=calculate_profit(x1,x2,y,z,result1[i,j][5],result1[i,j][6],result1[i,j][7],p_product)
                                    result2[i,int((j-1)/2),count]=[x1,x2,y,z,profit,n_update,p1_update,p2_update]
                                    count+=1
            result2_value=np.zeros((8,8,16))
            for i in range(8):
                for j in range(8):
                    for k in range(16):
                        result2_value[i,j,k]=result2[i,j,k][4]
            result2_value_max=np.max(result2_value,axis=2)
            result1_value=np.zeros((8,16))
            for i in range(8):
                for j in range(16):
                    result1_value[i,j]=result1[i,j][4]
                    result1_value[:,1::2]=result1_value[:,1::2]+result2_value_max
            result1_value_max=np.max(result1_value,axis=1)
            final_result=[0 for i in range(16)]
            for i in range(16):
                final_result[i]=result[i][4]
                final_result[1::2]=final_result[1::2]+result1_value_max
            max_value=max(final_result)
            max_index = final_result.index(max_value)
            print(S)
            print(f"最终的最大利润为{max_value}")
            print("第一阶段的生产决策为:")
            print(result[max_index])
            if max_index%2:
                max_index_1 = np.argmax(result1_value[int((max_index-1)/2)])
                print("第二阶段的生产决策为:")
                print(result1[int((max_index-1)/2),max_index_1])
            if max_index_1%2:
                max_index_2 = np.argmax(result2_value[int((max_index-1)/2),int((max_index_1-1)/2)])
                print("第三阶段的生产决策为:")
                print(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2])
            if (result[max_index][0:4]==[0,0,1,1])&(result1[int((max_index-1)/2),max_index_1][0:4]==[0,0,1,1])&(result2[int((max_index-1)/2),int((max_index_1-1)/2),max_index_2][0:4]==[1,1,0,0]):
                same+=1
            else:
                different+=1

1
最终的最大利润为33527367.088320002
第一阶段的生产决策为:
[0, 0, 0, 1, 298606.0800000001, 2213.0, 0.08000000000000002, 0.08000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 66081.525504, 489.0, 0.08000000000000002, 0.08000000000000002]
第三阶段的生产决策为:
[0, 0, 1, 0, 16922.592192000004, 0, 0.08000000000000002, 0.08000000000000002]
2
最终的最大利润为35560068.58136002
第一阶段的生产决策为:
[0, 0, 0, 1, 290903.84, 2297.0, 0.08000000000000002, 0.08000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 66820.61204800001, 527.0, 0.08000000000000002, 0.08000000000000002]
第三阶段的生产决策为:
[0, 0, 1, 0, 17987.850688000002, 0, 0.08000000000000002, 0.08000000000000002]
3
最终的最大利润为37650869.62240002
第一阶段的生产决策为:
[0, 0, 0, 1, 283201.60000000003, 2382.0, 0.08000000000000002, 0.08000000000000002]
第二阶段的生产决策为:
[0, 0, 0, 1, 67458.62112000001, 567.0, 0.08000000000000002, 0.08000000000000002]
第三阶段的生产决策为:
[0, 0, 1, 0, 19084.40352, 0, 0.08000000000000002, 0.08000000000000002]
4
最终的最大利润为39730688.047919944
第一阶段的生产决策为:
[0, 0, 0, 1, 275499.36, 2467.0, 0.08000000000000002, 0.0800000

In [8]:
print(f"在0.8-1.2波动时,生产决策方案和原方案一致的次数为:{same}")
print(f"在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:{different}")
print(f"在0.8-1.2波动时,决策差异率为:{different/125}")

在0.8-1.2波动时,生产决策方案和原方案一致的次数为:0
在0.8-1.2波动时,生产决策方案和原方案不一致的次数为:125
在0.8-1.2波动时,决策差异率为:1.0
